In [2]:
import torch
import torchvision

import torch.nn as nn
import numpy as np
import torch.nn.functional as F

from torch.optim import SGD

In [3]:
class SMSClassificationSoftMaxModel(nn.Module):
    def __init__(self, input_size):
        super().__init__()

        self.linear = nn.Linear(input_size, 3)

    def forward(self, x):
        x = self.linear(x)
        x = F.softmax(x)

        return x

In [4]:
vocabulary_size = 0
word2location = {}

def prepare_vocabulary(data):
    idx = 0
    for sentence in data:
        for word in sentence.split():
            if word not in word2location:
                word2location[word] = idx
                idx += 1
    return idx

def convert2vec(sentence):
    res_vec = np.zeros(vocabulary_size)
    for word in sentence.split():
        if word in word2location:
            res_vec[word2location[word]] += 1

    return res_vec

data = ["Your auto payment of 50$ was charged successfully!",
        "You have received a refund of 20$",
        "Please complete your power point presentation by tomorrow.",
        "You must arrive on time tomorrow, otherwise the manager will want to talk to you",
        "How about celebrating Bob's birthday party next week?",
        "Everyone is waiting for you at home, please come home early!"]

vocabulary_size = prepare_vocabulary(data)

In [5]:
inputs = torch.FloatTensor([convert2vec(data[i]) for i in range(len(data))])
labels = torch.FloatTensor([[1, 0, 0], [1, 0, 0], [0, 1, 0], [0, 1, 0], [0, 0, 1], [0, 0, 1]])

C:\Users\Dima\AppData\Local\Temp\ipykernel_19180\151121362.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:277.)
  inputs = torch.FloatTensor([convert2vec(data[i]) for i in range(len(data))])


In [10]:
# model
model = SMSClassificationSoftMaxModel(vocabulary_size)

# loss
loss_fn = nn.CrossEntropyLoss()

# Optimizer
optimizer = SGD(model.parameters(), lr=0.00001)

for i in range(50000):
    # Zero your gradients for every batch!
    optimizer.zero_grad()

    # output of model
    outputs = model(inputs)
    
    loss = loss_fn(outputs, labels)
    loss.backward()

    optimizer.step()
    
    if i % 200 == 0:
        print(i)
        print(loss.item())
        print('---------')

C:\Users\Dima\AppData\Local\Temp\ipykernel_19180\3516789844.py:9: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.softmax(x)


0
1.117691993713379
---------
200
1.117448091506958
---------
400
1.1172040700912476
---------
600
1.1169599294662476
---------
800
1.116715908050537
---------
1000
1.1164716482162476
---------
1200
1.1162269115447998
---------
1400
1.1159822940826416
---------
1600
1.1157375574111938
---------
1800
1.1154924631118774
---------
2000
1.115247130393982
---------
2200
1.1150015592575073
---------
2400
1.1147558689117432
---------
2600
1.1145100593566895
---------
2800
1.1142641305923462
---------
3000
1.1140180826187134
---------
3200
1.1137721538543701
---------
3400
1.1135259866714478
---------
3600
1.1132797002792358
---------
3800
1.1130331754684448
---------
4000
1.1127866506576538
---------
4200
1.1125396490097046
---------
4400
1.1122925281524658
---------
4600
1.112045168876648
---------
4800
1.1117979288101196
---------
5000
1.1115505695343018
---------
5200
1.1113032102584839
---------
5400
1.1110554933547974
---------
5600
1.1108078956604004
---------
5800
1.1105601787567139
--

In [9]:
print(model(torch.FloatTensor(convert2vec("Your payment has been received, no refund is currently available."))))
print(model(torch.FloatTensor(convert2vec("The manager said that your presentation went well, but next time make sure to arrive on time"))))
print(model(torch.FloatTensor(convert2vec("We are all waiting for you at the birthday party!"))))

tensor([0.2832, 0.3342, 0.3827], grad_fn=<SoftmaxBackward0>)
tensor([0.1988, 0.4658, 0.3353], grad_fn=<SoftmaxBackward0>)
tensor([0.3428, 0.3141, 0.3430], grad_fn=<SoftmaxBackward0>)


C:\Users\Dima\AppData\Local\Temp\ipykernel_19180\3516789844.py:9: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.softmax(x)
